<a href="https://colab.research.google.com/github/deepukr007/fashion-MNIST/blob/main/fashion_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from torch.autograd import Variable


In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
train_set = torchvision.datasets.FashionMNIST("./data", download=True, transform=transforms.Compose([transforms.ToTensor()]))
test_set = torchvision.datasets.FashionMNIST("./data", download=True, train=False, transform=transforms.Compose([transforms.ToTensor()]))
train_loader = torch.utils.data.DataLoader(train_set, batch_size=50)
test_loader = torch.utils.data.DataLoader(test_set,batch_size=50)

100%|██████████| 26421880/26421880 [00:01<00:00, 16133025.32it/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 272747.44it/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 4992877.21it/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 25828082.53it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



In [35]:
#printing out sizes

print("Train and test set lenghts: ")
print(len(train_set))
print(len(test_set))
print()
print("Batch size")
print(next(iter(train_loader))[0].size())


Train and test set lenghts: 
60000
10000

Batch size
torch.Size([50, 1, 28, 28])


In [22]:
class net(nn.Module):

    def __init__(self):
        super(net, self).__init__()

        self.cbrm1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.cbrm2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.fc1 = nn.Linear(in_features=128*6*6, out_features=256)
        self.fc2 = nn.Linear(in_features=256, out_features=10)

    def forward(self, x):
        out = self.cbrm1(x)
        out = self.cbrm2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = nn.functional.relu(out)
        out = self.fc2(out)


        return out

In [25]:
model = net()
model.to(device)

error = nn.CrossEntropyLoss()

learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
print(model)

net(
  (cbrm1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (cbrm2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=4608, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=10, bias=True)
)


In [56]:
#Network Training

num_epochs = 10
count = 0


# Accuracy - Classwise
predictions_list = []
labels_list = []

acuuracy =0
running_loss =0

for epoch in range(num_epochs):
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        train = images.view(50, 1, 28, 28)
        labels = labels

        # Forward pass
        outputs = model(train)
        loss = error(outputs, labels)

        # make gradients zero
        optimizer.zero_grad()

        #Back Propagation
        loss.backward()

        # Optimizer step
        optimizer.step()

        count += 1
        running_loss += loss.item()


        if not (count % 500):
          correct = 0
          total = 0
          for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            labels_list.append(labels)
            test = images.view(50, 1, 28, 28)
            outputs = model(test)
            predictions = torch.max(outputs, 1)[1].to(device)
            predictions_list.append(predictions)
            correct += (predictions == labels).sum()
            total += len(labels)
            accuracy = correct * 100 / total
            print("Iteration: {}, Loss: {}, Accuracy: {}%".format(count, running_loss/500, accuracy))
            running_loss = 0

Iteration: 500, Loss: 0.016475956467420474, Accuracy: 86.0%
Iteration: 500, Loss: 0.016475956467420474, Accuracy: 91.0%
Iteration: 500, Loss: 0.016475956467420474, Accuracy: 92.0%
Iteration: 500, Loss: 0.016475956467420474, Accuracy: 93.0%
Iteration: 500, Loss: 0.016475956467420474, Accuracy: 93.20000457763672%
Iteration: 500, Loss: 0.016475956467420474, Accuracy: 93.33333587646484%
Iteration: 500, Loss: 0.016475956467420474, Accuracy: 92.85713958740234%
Iteration: 500, Loss: 0.016475956467420474, Accuracy: 92.25%
Iteration: 500, Loss: 0.016475956467420474, Accuracy: 92.22222137451172%
Iteration: 500, Loss: 0.016475956467420474, Accuracy: 91.80000305175781%
Iteration: 500, Loss: 0.016475956467420474, Accuracy: 92.18181610107422%
Iteration: 500, Loss: 0.016475956467420474, Accuracy: 92.16667175292969%
Iteration: 500, Loss: 0.016475956467420474, Accuracy: 92.30769348144531%
Iteration: 500, Loss: 0.016475956467420474, Accuracy: 92.0%
Iteration: 500, Loss: 0.016475956467420474, Accuracy: 9

KeyboardInterrupt: ignored

In [55]:
correct = 0
total = 0
for images, labels in test_loader:
      images, labels = images.to(device), labels.to(device)
      labels_list.append(labels)
      test = images.view(50, 1, 28, 28)
      outputs = model(test)
      predictions = torch.max(outputs, 1)[1].to(device)
      predictions_list.append(predictions)
      correct += (predictions == labels).sum()

      total += len(labels)

      accuracy = correct * 100 / total
